In [ ]:
from pathlib import Path
import nltk as n
import os
from datetime import datetime
startTime = datetime.now()

cwd = str(Path.home())
masterloc=cwd+"/moviereviews/masterfile.txt"
stopwords = n.corpus.stopwords.words('english')
total_set=[]
test_set=[]
dev_set=[]
train_set=[]

def machine_features(characters,pos_tagged_s,reviewlist,sn,reviewSentence_list):
    #print("in machine feature")
    
    tot_features_dic={}
    #tot_features_dic.update(char_feature(characters,reviewlist))
    #tot_features_dic.update(verb_feature(pos_tagged_s,reviewlist))
    tot_features_dic.update(sentenceVerb_features(characters,pos_tagged_s,reviewSentence_list))
    total_set.append((tot_features_dic,sn))
        
def sentenceVerb_features(characters,pos_tagged_s,reviewSentence_list):
    VB_list=["VB","VBD","VBG","VBN","VBP","VBZ"]
    verblist_s=[x for x,y in pos_tagged_s if y in VB_list]
    total_sentences = len(reviewSentence_list)
    tot_occ_char_verb = 0
    for i in reviewSentence_list:
        if(len(i)!=0):
            v_bool=False
            c_bool=False
            words = n.tokenize.word_tokenize(i)
            pos_tagged_r = n.pos_tag(words)
            pos_tagged_r=[(w,l) for w,l in pos_tagged_r if w not in stopwords]
            for x in characters:
                x=x.lower()
                if x in words:
                    c_bool=True
                    break        
            for x,y in pos_tagged_r:
                if y in VB_list and x in verblist_s:
                    v_bool=True
                    break
            if c_bool and v_bool:
                tot_occ_char_verb = tot_occ_char_verb + 1
    per_occ_char_verb = tot_occ_char_verb/total_sentences
    return({"percentage_of_occurrence_character_verb":per_occ_char_verb})
    
def char_feature(characters,reviewlist):
    tot_occ_char=0
    per_tot_occ_char=0
    if len(reviewlist)!=0:
        for i in characters:
            i=i.lower()
            tot_occ_char+=reviewlist.count(i)
        per_tot_occ_char=tot_occ_char/len(reviewlist)
    return ({"percentage_of_occurence_character":per_tot_occ_char})

def verb_feature(pos_tagged_s,reviewlist):
    #print("in verb feature")
    pos_tagged_r=n.pos_tag(reviewlist)
    pos_tagged_r=[(w,l) for w,l in pos_tagged_r if w not in stopwords]
    tot_occ_verb=0
    per_tot_occ_verb=0
    VB_list=["VB","VBD","VBG","VBN","VBP","VBZ"]
    verblist_s=[x for x,y in pos_tagged_s if y in VB_list]
    verblist_r=[x for x,y in pos_tagged_r if y in VB_list]
    if len(reviewlist)!=0:
        for i in verblist_r:
            if i in verblist_s:
                tot_occ_verb+=synopsislist.count(i)
        per_tot_occ_verb=tot_occ_verb/len(reviewlist)
    return ({"percentage_of_occurence_verb":per_tot_occ_verb})

    
def trainmachine(t_set):
    return n.NaiveBayesClassifier.train(train_set)

def testmachine(machine,t_set):
    print("accuracy of the machine is :",n.classify.accuracy(machine,t_set))
    
f1 = open(masterloc)
count=0
stopwords
for i in f1.readlines():
    count+=1
    #print(count,end=",")
    moviehash=i.rstrip("\n")
    movie_loc=cwd+"/moviereviews/"+moviehash


    movie_n_loc=movie_loc+"/"+moviehash+".txt"
    #movie_n=open(movie_n_loc).readline()


    movie_s_loc=movie_loc+"/"+moviehash+"_s.txt"
    movie_s=open(movie_s_loc).read()

    
    synopsislist=n.word_tokenize(movie_s)                 # pos tagging synopsis early to avoid it doing again and again
    pos_tagged_s=n.pos_tag(synopsislist)
    pos_tagged_s=[(w,l) for w,l in pos_tagged_s if w not in stopwords]

    movie_c_loc=movie_loc+"/"+moviehash+"_c.txt"
    movie_c=[j.rstrip("\n") for j in open(movie_c_loc).readlines()]

    for i in range(25):
        movie_r_loc=movie_loc+"/"+moviehash+"_"+str(i+1)+".txt"
        if os.path.isfile(movie_r_loc)==True:
            f=open(movie_r_loc)
            reviewnumber,sn=f.readline().rstrip("\n").split(",")
            review=f.readline()
            review=review.lower()
            if review!="":
                reviewSentence_list=n.tokenize.sent_tokenize(review)
                reviewlist=n.tokenize.word_tokenize(review)
                machine_features(movie_c,pos_tagged_s,reviewlist,sn,reviewSentence_list)
            f.close()
f1.close()
            
tt=len(total_set)
#train_set=total_set[:int(0.4*tt)]
#dev_set=total_set[int(0.4*tt)+1:int(0.7*tt)]
#test_set=total_set[int(0.7*tt)+1:]
    
print(" ")
test_set=total_set[:int(0.3*tt)]
train_set=total_set[int(0.3*tt)+1:int(0.7*tt)]
dev_set=total_set[int(0.7*tt)+1:]
        
    
machine=trainmachine(train_set)
testmachine(machine,test_set)
testmachine(machine,dev_set)



machine.show_most_informative_features()
print(datetime.now() - startTime)

In [8]:
count=0
right=0
wrong=0
f1 = open(masterloc)
for i in f1.readlines():
    moviehash=i.rstrip("\n")
    movie_loc=cwd+"/moviereviews/"+moviehash
    
    
    movie_n_loc=movie_loc+"/"+moviehash+".txt"
    movie_n=open(movie_n_loc).readline()
    
    
    movie_s_loc=movie_loc+"/"+moviehash+"_s.txt"
    movie_s=open(movie_s_loc).read()
    
    
    movie_c_loc=movie_loc+"/"+moviehash+"_c.txt"
    movie_c=[j.rstrip("\n") for j in open(movie_c_loc).readlines()]
    
    for i in range(25):
        
        movie_r_loc=movie_loc+"/"+moviehash+"_"+str(i+1)+".txt"
        if os.path.isfile(movie_r_loc)==True:
            f=open(movie_r_loc)
            reviewnumber,sn=f.readline().rstrip("\n").split(",")
            review=f.readline().lower()
            f.close()
            if review!="":
                t=machine.classify(char_feature(movie_c,review)) 
                count+=1
            if t==sn:
                right+=1
            else:
                wrong+=1
                
f1.close()


print(right/count)
print(wrong/count)


0.7410536383199171
0.2594245636407109


In [5]:
train_set=total_set[:int(0.4*tt)]
dev_set=total_set[int(0.4*tt)+1:int(0.7*tt)]
test_set=total_set[int(0.7*tt)+1:]

testmachine(machine,test_set)
testmachine(machine,dev_set)


accuracy of the machine is : 0.6989904357066951
accuracy of the machine is : 0.7640180706882806
